<a href="https://colab.research.google.com/github/raquelcarmo/tropical_cyclones/blob/import-py-files/src/code/TC_GradCAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports and configurations

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/ESRIN_PhiLab/Tropical_Cyclones/tropical_cyclones/src/code
%ls

import imp 
# import helper.py
helper = imp.new_module('helper_functions')
exec(open("./helper_functions.py").read(), helper.__dict__)
# import models.py
models = imp.new_module('models')
exec(open("./models.py").read(), models.__dict__)
# import data_processor.py
dp = imp.new_module('data_processor')
exec(open("./data_processor.py").read(), dp.__dict__)

In [ ]:
# getting in the directory 
%cd /content/drive/My Drive/ESRIN_PhiLab/Tropical_Cyclones/data
%ls

In [ ]:
# general imports
import random
import glob
import os
import sys
sys.stdout.flush()
import pandas as pd
import numpy as np
from keras import backend as K
import cv2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math
import imageio
import os.path
from PIL import Image
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy import ndimage
from google.colab.patches import cv2_imshow
import random
from shapely.geometry import Point
import re
import pickle
import scipy
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.data import Dataset
from tensorflow.keras import Input
from tensorflow.keras.applications import resnet50, mobilenet_v2
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import concatenate, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, TruePositives, FalsePositives, TrueNegatives, FalseNegatives

np.set_printoptions(precision=4)

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch" of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):
    # First, we create a model that maps the input image to the activations of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

Load previously saved model.

In [ ]:
main_dir = "SAR_swath_images_VV+VH+WS/"
dir = main_dir + "classification_results/categorization/ResNet_Numeric-F_BatchSize-8_lr-0.0001_Epochs-20_Folds-5_Norm-T_Aug-T/"
#dir = main_dir + "classification_results/identification/ResNet_Numeric-F_BatchSize-8_lr-0.0001_Epochs-20_Folds-5_Norm-T/"
#save_path = dir + "Gradcam_heatmaps"
#os.makedirs(save_path, exist_ok=True)

# load previously saved model
model = tf.keras.models.load_model(dir + "model_4.h5", compile = False)
#model = tf.keras.models.load_model(dir + "best_model_fine_tuned_4.h5", compile = False)
model.summary()

In [ ]:
# source: https://stackoverflow.com/questions/50283844/in-keras-how-to-get-the-layer-name-associated-with-a-model-object-contained-i
last_conv_layer_name = model.get_layer('resnet50').layers[-1].name
last_conv_layer = model.get_layer('resnet50').get_layer(last_conv_layer_name)

print("last_conv_layer_name:", last_conv_layer_name)
print("last_conv_layer:", last_conv_layer)

In [ ]:
last_conv_layer.output

In [ ]:
model.get_layer('resnet50').summary()

In [ ]:
model.inputs

In [ ]:
model.get_layer("resnet50").inputs

In [ ]:
from tensorflow import keras
last_conv_layer_model = keras.Model(model.get_layer("resnet50").inputs, last_conv_layer.output)

In [ ]:
classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
x = classifier_input
classifier_layer_names = [
    "global_average_pooling2d",
    "dense"]
for layer_name in classifier_layer_names:
    x = model.get_layer(layer_name)(x)
classifier_model = keras.Model(classifier_input, x)


In [ ]:
model.get_layer('mobilenetv2_1.00_224').layers[-1].name
[layer.name for layer in model.layers[-2:]]

In [ ]:
last_conv_layer_name = "conv5_block3_out"

# check the names with model.summary(), sometimes recreating the model multiple times for testing the names can change
classifier_layer_names = [
    "global_average_pooling2d",
    "dense"
    #"global_average_pooling2d_1",
    #"dense_1"
]

Actual GradCam testing

In [ ]:
"""
MIN_HEIGHT = 700
MIN_WIDTH = 400
BATCH_SIZE = 1
NETWORK = "ResNet"

# Preprocess image
df = pd.read_csv(main_dir + "/csv/full_dataset.csv", converters={'bbox_shape': eval}).dropna()
images, labels, bboxes = helper.load_from_df(df)

# create an instance of the DataProcessor
processor = DataProcessor(model = NETWORK,
                          min_height = MIN_HEIGHT,
                          min_width = MIN_WIDTH)

processed_image = np.array([processor.preprocess_pipeline(images[2], bboxes[2])])

# Print what the top predicted class is
preds = model.predict(processed_image)
#print("Predicted:", decode_predictions(preds, top=1)[0])
print(preds)
print(labels[2])
# Generate class activation heatmap
heatmap = make_gradcam_heatmap(
    processed_image, model, last_conv_layer_name, classifier_layer_names
)

# Display heatmap
plt.matshow(heatmap)
plt.show()
sys.exit("Stop")
############################################
###  SUPERIMPOSED GARDCAM VISUALIZATION  ###
############################################

# Load the original image
img = keras.preprocessing.image.load_img(images[0])
img = keras.preprocessing.image.img_to_array(img)

# Rescale heatmap to a range 0-255
heatmap = np.uint8(255 * heatmap)

# Use jet colormap to colorize heatmap
jet = cm.get_cmap("jet")

# Use RGB values of the colormap
jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[heatmap]

# Create an image with RGB colorized heatmap
jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

# Superimpose the heatmap on original image
superimposed_img = jet_heatmap * 0.4 + img
superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

# Save the superimposed image
#superimposed_img.save(save_path)

# Display Grad CAM
display(Image(save_path))
"""

In [ ]:
# ATTENTION: to use this code, make sure that processor has:
#    #image_path = image_path.numpy().decode('utf-8')
#    #bbox = bbox.numpy()
#    image = cv2.imread(image_path)#.astype(np.float32) # loads images as BGR in float32
#    #image = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)   # BGR -> RGB
#    return sized_image, output_image 

MIN_HEIGHT = 700
MIN_WIDTH = 400
BATCH_SIZE = 1
NETWORK = "ResNet"
categorization = True

# Preprocess image
csv_path = main_dir + "/csv/full_dataset.csv"
images, labels, bboxes = helper.load_data(csv_path, with_cat = categorization)

# create an instance of the DataProcessor
processor = dp.DataProcessor(model = NETWORK,
                          min_height = MIN_HEIGHT,
                          min_width = MIN_WIDTH,
                          normalise = True)

count = -1
for image, bbox, label in zip(images, bboxes, labels):
    count += 1

    if count == 204:
        orig_image, processed_image = processor.preprocess_pipeline(image, bbox)
        #orig_image = np.transpose(orig_image,(1,0,2))
        orig_image = tf.transpose(orig_image, [1,0,2])

        processed_image = tf.transpose(processed_image, [1,0,2])
        processed_image = tf.expand_dims(processed_image, axis=0)
        #processed_image = np.array([processed_image])
        #processed_image = np.expand_dims(processed_image, axis=0)

        # Print what the top predicted class is
        preds = model.predict(processed_image)
        #print("Predicted:", decode_predictions(preds, top=1)[0])
        predictions = np.round(preds[0],2) if categorization else preds[0]
        print("Prediction: {}, True label: {}".format(predictions, label))

        # Generate class activation heatmap
        heatmap = make_gradcam_heatmap(processed_image, model, last_conv_layer_name, classifier_layer_names)

        # Display heatmap
        #plt.matshow(heatmap)
        #plt.show()
        #plt.imshow(orig_image[:,:,1])
        #plt.show()
        #plt.imshow(orig_image[:,:,2])
        #plt.show()

        ############################################
        ###  SUPERIMPOSED GARDCAM VISUALIZATION  ###
        ############################################
        # Load the original image
        #plt.imshow(orig_image)
        #plt.title("Original Image")
        #plt.show()
        #img = keras.preprocessing.image.load_img(image)
        #img = keras.preprocessing.image.img_to_array(img)

        # Rescale heatmap to a range 0-255
        heatmap = np.uint8(255 * heatmap)

        # Use jet colormap to colorize heatmap
        jet = cm.get_cmap("jet")

        # Use RGB values of the colormap
        jet_colors = jet(np.arange(256))[:, :3]
        jet_heatmap = jet_colors[heatmap]

        # Create an image with RGB colorized heatmap
        jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
        jet_heatmap = jet_heatmap.resize((orig_image.shape[1], orig_image.shape[0]))
        jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)
        jet_heatmap_img = keras.preprocessing.image.array_to_img(jet_heatmap)

        # Superimpose the heatmap on original image
        vv_channel = np.dstack((orig_image[:,:,2], orig_image[:,:,2], orig_image[:,:,2]))
        vh_channel = np.dstack((orig_image[:,:,1], orig_image[:,:,1], orig_image[:,:,1]))
        superimposed_img = jet_heatmap * 0.4 + vv_channel
        superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

        # Save the superimposed image
        #superimposed_img.save(save_path)

        # Display Grad CAM
        #display(Image(save_path))
        #plt.imshow(superimposed_img)
        #plt.title("Superimposed Image")
        #plt.show()

        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize = (15,9.2))
        font_size = 18
        ax1.imshow(vv_channel)    # plot vv channel
        ax1.set_title("Original Image (VV) - label: {}".format(label), fontsize = font_size)

        #ax2.imshow(np.squeeze(processed_image, axis=0))
        #ax2.set(title = "Normalised Image")
        ax2.imshow(vh_channel)  # plot vh channel
        ax2.set_title("Original Image (VH)", fontsize = font_size)

        #ax3.matshow(heatmap)
        #ax3.set(title = "Heatmap - prediction: {}".format(predictions))
        heat_plot = ax3.imshow(jet_heatmap_img, cmap='jet')
        ax3.set_title("Heatmap - prediction: {}".format(predictions), fontsize = font_size)
        divider = make_axes_locatable(ax3)
        cax = divider.append_axes("right", size="5%", pad=0.1)
        plt.colorbar(heat_plot, cax=cax)

        ax4.imshow(superimposed_img)
        ax4.set_title("Superimposed Image", fontsize = font_size)
        fig.tight_layout()
        plt.show()
        
        #fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize = (15,12))
        #ax1.imshow(vv_channel)  # plot vv channel
        #ax1.set(title = "Original Image (VV) - label: {}".format(label))

        #ax2.imshow(vh_channel)  # plot vh channel
        #ax2.set(title = "Original Image (VH)")

        #heat_plot = ax3.imshow(jet_heatmap, cmap='jet')
        #ax3.set(title = "Heatmap - prediction: {}".format(predictions))
        #divider = make_axes_locatable(ax3)
        #cax = divider.append_axes("right", size="5%", pad=0.1)
        #plt.colorbar(heat_plot, cax=cax)

        #ax4.imshow(superimposed_img)
        #ax4.set(title = "Superimposed Image")
        #fig.tight_layout()
        #plt.show()

        #fig.savefig("{}/id_resnet_gradcam.jpg".format(save_path))
        #fig.savefig("{}/superimposed_{}.jpg".format(save_path, count))
        fig.savefig("{}/superimposed_{}.jpg".format(dir, np.random.randint(low=1, high=100, size=1)[0]), box_layout='tight')

In [ ]:
save_path = dir + "gradcam_heatmaps_{}".format(fold_var)
os.makedirs(save_path, exist_ok=True)

def grad_cam(model, NETWORK, val_dataset, val_norm_dataset, save_path):
    cnt = 0
    if NETWORK == "ResNet":
        # check the names with model.summary()
        last_conv_layer_name = "conv5_block3_out"
        
        classifier_layer_names = [
            "global_average_pooling2d",
            "dense"
            ]
            
    else:  #MobileNetV2
        last_conv_layer_name = "out_relu" # have to check this again
        
        classifier_layer_names = [
            "global_average_pooling2d_1",
            "dense_1"
            ]

    for orig_image, label, processed_image, processed_label in zip(val_dataset, val_norm_dataset):
        # label and norm_label should be the same

        pred = model.predict(processed_image)
        prediction = np.round(pred[0],2) #if categorization else pred[0]

        heatmap = make_gradcam_heatmap(
            processed_image, model, last_conv_layer_name, classifier_layer_names
            )

        # Rescale heatmap to a range 0-255
        heatmap = np.uint8(255 * heatmap)

        # Use jet colormap to colorize heatmap
        jet = cm.get_cmap("jet")

        # Use RGB values of the colormap
        jet_colors = jet(np.arange(256))[:, :3]
        jet_heatmap = jet_colors[heatmap]

        # Create an image with RGB colorized heatmap
        jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
        jet_heatmap = jet_heatmap.resize((orig_image.shape[1], orig_image.shape[0]))
        jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

        # Superimpose the heatmap on original image
        vv_channel = np.dstack((orig_image[:,:,0], orig_image[:,:,0], orig_image[:,:,0]))
        vh_channel = np.dstack((orig_image[:,:,1], orig_image[:,:,1], orig_image[:,:,1]))
        superimposed_img = jet_heatmap * 0.4 + vv_channel
        superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

        # Display Grad CAM
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize = (15,9.2))
        font_size = 18
        ax1.imshow(vv_channel)    # plot vv channel
        ax1.set_title("Original Image (VV) - label: {}".format(label), fontsize = font_size)
        ax2.imshow(vh_channel)  # plot vh channel
        ax2.set_title("Original Image (VH)", fontsize = font_size)
        heat_plot = ax3.imshow(jet_heatmap, cmap='jet')
        ax3.set_title("Heatmap - prediction: {}".format(prediction), fontsize = font_size)
        divider = make_axes_locatable(ax3)
        cax = divider.append_axes("right", size="5%", pad=0.1)
        plt.colorbar(heat_plot, cax=cax)
        ax4.imshow(superimposed_img)
        ax4.set_title("Superimposed Image", fontsize = font_size)
        fig.tight_layout()
        #plt.show()

        fig.savefig("{}/heatmap_{}.jpg".format(save_path, cnt), bbox_inches='tight')
        cnt += 1

    return

In [ ]:
##################################
###          SETTINGS         ####
##################################
main_dir = "SAR_swath_images_VV+VH+WS_dilated"
NETWORK = "ResNet"     # options: ["Mobile", "ResNet", "VGG"]
MIN_HEIGHT = MIN_WIDTH = 384
EYE_ONLY = True
NUM_VARS = False
NORMALISE = True
NORM_MODE = "model" # options: ["model", "z-norm", "simple", "none"]
ROTATE = False
N_CROPS = 1
CROP_MODE = "uniform" # options: ["uniform", "weighted"]
AUGMENT = True
BATCH_SIZE = 8
BUFFER_SIZE = 100
SPLIT_NUMBER = 5
##################################

full_dataset_path = "{}/csv/full_dataset.csv".format(main_dir)
df = pd.read_csv(full_dataset_path, converters={'bbox_shape': eval}).dropna()
print("Dataset dimension: {}".format(len(df)))


# extract non-categorical labels for StratifiedKFold
Y = np.zeros(len(df), dtype=int)
for index, row in df.iterrows():
    if df["label"][index] != 0:
        cat = df["image"][index].split('/')[1]
        Y[index] = int(cat[-1])

if EYE_ONLY:
    assert isinstance(Y, np.ndarray)
    idx = np.where(Y == 0)[0].tolist()
    Y = np.delete(Y, idx)
    Y -= 1
    df.drop(idx, inplace = True)
    df.reset_index(drop=True, inplace=True)
    print("New dimensions, Y: {} and df: {}".format(len(Y), len(df)))


# create an instance of the DataProcessor
processor = dp.DataProcessor(model = NETWORK,
                            min_height = MIN_HEIGHT,
                            min_width = MIN_WIDTH,
                            normalise = NORMALISE,           # perform normalisation [DEPRECATED]
                            rotate = ROTATE,                 # perform rotation
                            plot_light = False,              # plot only select_crop() images
                            plot_extensive = False,          # plot extensively all images
                            show_prints = False)


print("Entering in k fold cross validation...")
stratified_k_fold = StratifiedKFold(n_splits = SPLIT_NUMBER, shuffle = False)
fold_var = 1

for train_index, val_index in stratified_k_fold.split(np.zeros(len(df)), Y):
    training_data = df.iloc[train_index]
    validation_data = df.iloc[val_index]

    # LOAD DATA
    train_images, train_labels, train_bbox = helper.load_data(df = training_data, eye_only = EYE_ONLY)
    val_images, val_labels, val_bbox = helper.load_data(df = validation_data, eye_only = EYE_ONLY)

    # GENERATE DATASETS
    train_dataset = helper.create_dataset(processor, train_images, train_labels, train_bbox, N_CROPS, CROP_MODE, MIN_HEIGHT, MIN_WIDTH)
    val_dataset = helper.create_dataset(processor, val_images, val_labels, val_bbox, 1, CROP_MODE, MIN_HEIGHT, MIN_WIDTH)

    # PERFORM NORMALISATION
    if NORMALISE:
        train_norm_dataset, val_norm_dataset = helper.normalisation(train_dataset, val_dataset, mode = NORM_MODE, model = NETWORK)
    else:
        train_norm_dataset = train_dataset
        val_norm_dataset = val_dataset
    unbatched_train_norm_dataset, unbatched_val_norm_dataset = train_norm_dataset, val_norm_dataset

    # CONFIGURE FOR PERFORMANCE
    #SQUARED = True if MIN_HEIGHT == MIN_WIDTH else False
    #train_norm_dataset = helper.configure_for_performance(train_norm_dataset, BUFFER_SIZE, BATCH_SIZE, shuffle = True, augment = AUGMENT, squared_input = SQUARED)
    val_norm_dataset = helper.configure_for_performance(val_norm_dataset, BUFFER_SIZE, BATCH_SIZE)

    # load previously saved model
    main_dir = "SAR_swath_images_VV+VH+WS_dilated/"
    dir = main_dir + "classification_results/categorization/test_eye_only/ResNet_nu-F_bs-8_384x384_lr-1e-05_ep-30_sp-5_no-mT_cr-u1_ag-T_drp-F/"
    model = tf.keras.models.load_model(dir + "model_2.h5", compile = False)

    # GRAD-CAM ANALYSIS
    predictions = model.predict(val_norm_dataset)
    helper.grad_cam(model, val_dataset, unbatched_val_norm_dataset, predictions, save_path = "")

    fold_var +=1
    if fold_var == 2:
        sys.exit()

##2. Test other grad-cam code

Source: https://github.com/eclique/keras-gradcam/blob/master/grad_cam.py

In [ ]:
def grad_cam(input_model, image, cls, layer_name):
    """GradCAM method for visualizing input saliency."""
    y_c = input_model.output[0, cls.numpy()]
    conv_output = input_model.get_layer(layer_name).output
    grads = K.gradients(y_c, conv_output)[0]
    # Normalize if necessary
    # grads = normalize(grads)
    gradient_function = K.function([input_model.input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    # Process CAM
    cam = cv2.resize(cam, (W, H), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam_max = cam.max() 
    if cam_max != 0: 
        cam = cam / cam_max
    return cam

In [ ]:
def compute_saliency(model, val_dataset, val_norm_dataset, predictions, layer_name='block5_conv3', cls=-1, visualize=True, save=False):
    """Compute saliency using all three approaches.
        -layer_name: layer to compute gradients;
        -cls: class number to localize (-1 for most probable class).
    """

    val_norm_images = val_norm_dataset.map(lambda x, y: x)
    val_norm_labels = val_norm_dataset.map(lambda x, y: y)
    val_images = val_dataset.map(lambda x, y: x)
    val_labels = val_dataset.map(lambda x, y: y)

    for orig_image, label, processed_image, processed_label, prediction in zip(val_images, val_labels, val_norm_images, val_norm_labels, predictions):
        # label and norm_label should be the same

        #predictions = model.predict(processed_image)

        #predictions = model.predict(preprocessed_input)
        #top_n = 5
        #top = decode_predictions(predictions, top=top_n)[0]
        #classes = np.argsort(predictions[0])[-top_n:][::-1]
        #print('Model prediction:')
        #for c, p in zip(classes, top):
        #    print('\t{:15s}\t({})\twith probability {:.3f}'.format(p[1], c, p[2]))
        #if cls == -1:
        #    cls = np.argmax(predictions)
        #class_name = decode_predictions(np.eye(1, 1000, cls))[0][0][1]
        #print("Explanation for '{}'".format(class_name))
        
        gradcam = grad_cam(model, processed_image, label, layer_name)
        #gb = guided_backprop(guided_model, preprocessed_input, layer_name)
        #guided_gradcam = gb * gradcam[..., np.newaxis]

        if save:
            jetcam = cv2.applyColorMap(np.uint8(255 * gradcam), cv2.COLORMAP_JET)
            jetcam = (np.float32(jetcam) + load_image(img_path, preprocess=False)) / 2
            cv2.imwrite('gradcam.jpg', np.uint8(jetcam))
            cv2.imwrite('guided_backprop.jpg', deprocess_image(gb[0]))
            cv2.imwrite('guided_gradcam.jpg', deprocess_image(guided_gradcam[0]))
        
        if visualize:
            plt.figure(figsize=(15, 10))
            #plt.subplot(131)
            plt.title('GradCAM')
            plt.axis('off')
            plt.imshow(load_image(img_path, preprocess=False))
            plt.imshow(gradcam, cmap='jet', alpha=0.5)

            #plt.subplot(132)
            #plt.title('Guided Backprop')
            #plt.axis('off')
            #plt.imshow(np.flip(deprocess_image(gb[0]), -1))
            
            #plt.subplot(133)
            #plt.title('Guided GradCAM')
            #plt.axis('off')
            #plt.imshow(np.flip(deprocess_image(guided_gradcam[0]), -1))
            #plt.show()
            
        return gradcam#, gb, guided_gradcam

In [ ]:
##################################
###          SETTINGS         ####
##################################
main_dir = "SAR_swath_images_VV+VH+WS_dilated"
NETWORK = "ResNet"     # options: ["Mobile", "ResNet", "VGG"]
MIN_HEIGHT = MIN_WIDTH = 384
EYE_ONLY = True
NUM_VARS = False
NORMALISE = True
NORM_MODE = "model" # options: ["model", "z-norm", "simple", "none"]
ROTATE = False
N_CROPS = 1
CROP_MODE = "uniform" # options: ["uniform", "weighted"]
AUGMENT = True
BATCH_SIZE = 8
BUFFER_SIZE = 100
SPLIT_NUMBER = 5
##################################

full_dataset_path = "{}/csv/full_dataset.csv".format(main_dir)
df = pd.read_csv(full_dataset_path, converters={'bbox_shape': eval}).dropna()
print("Dataset dimension: {}".format(len(df)))


# extract non-categorical labels for StratifiedKFold
Y = np.zeros(len(df), dtype=int)
for index, row in df.iterrows():
    if df["label"][index] != 0:
        cat = df["image"][index].split('/')[1]
        Y[index] = int(cat[-1])

if EYE_ONLY:
    assert isinstance(Y, np.ndarray)
    idx = np.where(Y == 0)[0].tolist()
    Y = np.delete(Y, idx)
    Y -= 1
    df.drop(idx, inplace = True)
    df.reset_index(drop=True, inplace=True)
    print("New dimensions, Y: {} and df: {}".format(len(Y), len(df)))


# create an instance of the DataProcessor
processor = dp.DataProcessor(model = NETWORK,
                            min_height = MIN_HEIGHT,
                            min_width = MIN_WIDTH,
                            normalise = NORMALISE,           # perform normalisation [DEPRECATED]
                            rotate = ROTATE,                 # perform rotation
                            plot_light = False,              # plot only select_crop() images
                            plot_extensive = False,          # plot extensively all images
                            show_prints = False)


print("Entering in k fold cross validation...")
stratified_k_fold = StratifiedKFold(n_splits = SPLIT_NUMBER, shuffle = False)
fold_var = 1

for train_index, val_index in stratified_k_fold.split(np.zeros(len(df)), Y):
    training_data = df.iloc[train_index]
    validation_data = df.iloc[val_index]

    # LOAD DATA
    train_images, train_labels, train_bbox = helper.load_data(df = training_data, eye_only = EYE_ONLY)
    val_images, val_labels, val_bbox = helper.load_data(df = validation_data, eye_only = EYE_ONLY)

    # GENERATE DATASETS
    train_dataset = helper.create_dataset(processor, train_images, train_labels, train_bbox, N_CROPS, CROP_MODE, MIN_HEIGHT, MIN_WIDTH)
    val_dataset = helper.create_dataset(processor, val_images, val_labels, val_bbox, 1, CROP_MODE, MIN_HEIGHT, MIN_WIDTH)

    # PERFORM NORMALISATION
    if NORMALISE:
        train_norm_dataset, val_norm_dataset = helper.normalisation(train_dataset, val_dataset, mode = NORM_MODE, model = NETWORK)
    else:
        train_norm_dataset = train_dataset
        val_norm_dataset = val_dataset
    unbatched_train_norm_dataset, unbatched_val_norm_dataset = train_norm_dataset, val_norm_dataset

    # CONFIGURE FOR PERFORMANCE
    #SQUARED = True if MIN_HEIGHT == MIN_WIDTH else False
    #train_norm_dataset = helper.configure_for_performance(train_norm_dataset, BUFFER_SIZE, BATCH_SIZE, shuffle = True, augment = AUGMENT, squared_input = SQUARED)
    val_norm_dataset = helper.configure_for_performance(val_norm_dataset, BUFFER_SIZE, BATCH_SIZE)

    # load previously saved model
    main_dir = "SAR_swath_images_VV+VH+WS_dilated/"
    dir = main_dir + "classification_results/categorization/test_eye_only/ResNet_nu-F_bs-8_384x384_lr-1e-05_ep-30_sp-5_no-mT_cr-u1_ag-T_drp-F/"
    model = tf.keras.models.load_model(dir + "model_2.h5", compile = False)

    # GRAD-CAM ANALYSIS
    predictions = model.predict(val_norm_dataset)
    compute_saliency(model, val_dataset, unbatched_val_norm_dataset, predictions, layer_name='block5_conv3', cls=-1, visualize=True, save=False)

    fold_var +=1
    if fold_var == 2:
        sys.exit()